In [1]:
import numpy as np 
import pandas as pd 
import tensorflow as tf
import matplotlib.pyplot as plt

In [2]:
train=pd.read_csv('../input/titanic/train.csv')
test=pd.read_csv("../input/titanic/test.csv")
df=[train,test]

In [3]:
test['Survived']=np.nan

In [4]:
train.info()
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass  

In [5]:
for data in df:
#     print(data.isnull().sum())
#     data['Age'].fillna(data['Age'].mean(),inplace=True)
    data['Embarked'].fillna(data['Embarked'].mode()[0],inplace=True)
    data['Fare'].fillna(data['Fare'].mean(),inplace=True)
    data['Cabin']=data['Cabin'].fillna("Unknown")
    data['Deck']=data['Cabin'].str.get(0)

# Feature Engineering

In [6]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
for data in df:
    data['Sex']=le.fit_transform(data['Sex'])
    
    data['Embarked']=le.fit_transform(data['Embarked'])
    
    data['Name']=data['Name'].apply(lambda x:x.split(',')[1].split('.')[0].strip())
    Title_Dict = {}
    Title_Dict.update(dict.fromkeys(['Capt', 'Col', 'Major', 'Dr', 'Rev'], 'Officer'))
    Title_Dict.update(dict.fromkeys(['Don', 'Sir', 'the Countess', 'Dona', 'Lady'], 'Royalty'))
    Title_Dict.update(dict.fromkeys(['Mme', 'Ms', 'Mrs'], 'Mrs'))
    Title_Dict.update(dict.fromkeys(['Mlle', 'Miss'], 'Miss'))
    Title_Dict.update(dict.fromkeys(['Mr'], 'Mr'))
    Title_Dict.update(dict.fromkeys(['Master','Jonkheer'], 'Master'))
    data['Title'] = data['Name'].map(Title_Dict)
    data['Title']=le.fit_transform(data['Title'])
    
    data['Lastname']=data['Name'].apply(lambda x: str.split(x, ",")[0])

    data['Age'] = data.groupby('Title')['Age'].transform(lambda x: x.fillna(x.median()))
    
#     data['FareBin'] = pd.qcut(data['Fare'], 5)
#     data['FareBin']=le.fit_transform(data['FareBin'])
    
    data['Fare']=(data['Fare']-data['Fare'].min())/(data['Fare'].max()-data['Fare'].min())

    data['FamilySize']=data['Parch']+data['SibSp']+1
    
    data['TravelAlone']=np.where(data['FamilySize']>1, 0, 1)
    
    data['Deck']=le.fit_transform(data['Deck'])
    
#     data['AgeBin'] = pd.qcut(data['Age'], 4)
#     data['AgeBin']=le.fit_transform(data['AgeBin'])
    
#     Ticket_Count = dict(data['Ticket'].value_counts())
#     data['TicketGroup'] = data['Ticket'].apply(lambda x:Ticket_Count[x])#.apply(label)
    

In [7]:
train.info()
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 17 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    int64  
 5   Age          891 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        891 non-null    object 
 11  Embarked     891 non-null    int64  
 12  Deck         891 non-null    int64  
 13  Title        891 non-null    int64  
 14  Lastname     891 non-null    object 
 15  FamilySize   891 non-null    int64  
 16  TravelAlone  891 non-null    int64  
dtypes: float64(2), int64(11), object(4)
memory usage: 118.5+ KB
<class 'pandas.core.frame.DataFr

In [8]:
X= pd.concat([train, test])

In [9]:
DEFAULT_SURVIVAL_VALUE = 0.5
X['Family_Survival'] = DEFAULT_SURVIVAL_VALUE

for grp, grp_df in X[['Survived','Name', 'Lastname', 'Fare', 'Ticket', 'PassengerId',
                           'SibSp', 'Parch', 'Age', 'Cabin']].groupby(['Lastname', 'Fare']):
    
    if (len(grp_df) != 1):
        # A Family group is found.
        for ind, row in grp_df.iterrows():
            smax = grp_df.drop(ind)['Survived'].max()
            smin = grp_df.drop(ind)['Survived'].min()
            passID = row['PassengerId']
            if (smax == 1.0):
                X.loc[X['PassengerId'] == passID, 'Family_Survival'] = 1
            elif (smin==0.0):
                X.loc[X['PassengerId'] == passID, 'Family_Survival'] = 0

for _, grp_df in X.groupby('Ticket'):
    if (len(grp_df) != 1):
        for ind, row in grp_df.iterrows():
            if (row['Family_Survival'] == 0) | (row['Family_Survival']== 0.5):
                smax = grp_df.drop(ind)['Survived'].max()
                smin = grp_df.drop(ind)['Survived'].min()
                passID = row['PassengerId']
                if (smax == 1.0):
                    X.loc[X['PassengerId'] == passID, 'Family_Survival'] = 1
                elif (smin==0.0):
                    X.loc[X['PassengerId'] == passID, 'Family_Survival'] = 0
                        

In [10]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1309 entries, 0 to 417
Data columns (total 18 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   PassengerId      1309 non-null   int64  
 1   Survived         891 non-null    float64
 2   Pclass           1309 non-null   int64  
 3   Name             1309 non-null   object 
 4   Sex              1309 non-null   int64  
 5   Age              1309 non-null   float64
 6   SibSp            1309 non-null   int64  
 7   Parch            1309 non-null   int64  
 8   Ticket           1309 non-null   object 
 9   Fare             1309 non-null   float64
 10  Cabin            1309 non-null   object 
 11  Embarked         1309 non-null   int64  
 12  Deck             1309 non-null   int64  
 13  Title            1309 non-null   int64  
 14  Lastname         1309 non-null   object 
 15  FamilySize       1309 non-null   int64  
 16  TravelAlone      1309 non-null   int64  
 17  Family_Survival

In [11]:
train=X[0:len(train)]
test=X[len(train):]
train=train.drop(['PassengerId','Ticket',"Cabin",'Name','SibSp','Parch','Age','Fare','Lastname'],axis='columns')
test=test.drop(['PassengerId','Survived','Ticket',"Cabin",'Name','SibSp','Parch','Age','Fare','Lastname'],axis='columns')
train.info()
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 0 to 890
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Survived         891 non-null    float64
 1   Pclass           891 non-null    int64  
 2   Sex              891 non-null    int64  
 3   Embarked         891 non-null    int64  
 4   Deck             891 non-null    int64  
 5   Title            891 non-null    int64  
 6   FamilySize       891 non-null    int64  
 7   TravelAlone      891 non-null    int64  
 8   Family_Survival  891 non-null    float64
dtypes: float64(2), int64(7)
memory usage: 69.6 KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 418 entries, 0 to 417
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Pclass           418 non-null    int64  
 1   Sex              418 non-null    int64  
 2   Embarked         418 non-null    int64  
 3   

In [13]:
train.sample(10)

,Survived,Pclass,Sex,Embarked,Deck,Title,FamilySize,TravelAlone,Family_Survival
750,1.0,2,0,2,8,1,3,0,1.0
97,1.0,1,1,0,3,2,2,0,0.5
416,1.0,2,0,2,8,3,3,0,0.5
515,0.0,1,1,2,3,2,1,1,0.5
517,0.0,3,1,1,8,2,1,1,1.0
337,1.0,1,0,0,4,1,1,1,1.0
93,0.0,3,1,2,8,2,4,0,1.0
221,0.0,2,1,2,8,2,1,1,1.0
855,1.0,3,0,2,8,3,2,0,0.5
482,0.0,3,1,2,8,2,1,1,1.0


In [14]:
X=train.drop(['Survived'],axis='columns').values
Y=train['Survived'].values
# from sklearn.model_selection import train_test_split
# x_train,x_test,y_train,y_test=train_test_split(X,Y,test_size=0.2)

# Model

In [15]:
from sklearn.model_selection import StratifiedShuffleSplit
sss = StratifiedShuffleSplit(n_splits=5, test_size=0.2)

In [16]:
from sklearn.ensemble import RandomForestClassifier
base_model=RandomForestClassifier()
base_model.fit(X,Y)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [17]:
pred=base_model.predict(X)
# pred=[int(round(x[0])) for x in pred]
# print(pred[0:10])
from sklearn.metrics import confusion_matrix
print(confusion_matrix(Y,pred))
from sklearn.metrics import accuracy_score
accuracy_score(Y,pred)

[[521  28]
 [ 70 272]]


0.8900112233445566

# Hyperparameter Tuning

In [18]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV

random_grid = {
                 'bootstrap': [True, False],
 'max_depth': [2,5,7,10,12,15],
 'max_features': [3, 'sqrt','log2'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [1,2,3,4],
 'n_estimators': [50,100,150,200,250,300,350],
 "criterion": ["gini", "entropy"]
             }

In [19]:
clf=RandomForestClassifier()
random_model=RandomizedSearchCV(estimator = clf, param_distributions = random_grid,n_iter=700, cv = sss, verbose=2, n_jobs = -1)
random_model.fit(X,Y)

Fitting 5 folds for each of 700 candidates, totalling 3500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:    6.0s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:   26.9s
[Parallel(n_jobs=-1)]: Done 357 tasks      | elapsed:   58.3s
[Parallel(n_jobs=-1)]: Done 640 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done 1005 tasks      | elapsed:  2.6min
[Parallel(n_jobs=-1)]: Done 1450 tasks      | elapsed:  3.7min
[Parallel(n_jobs=-1)]: Done 1977 tasks      | elapsed:  5.1min
[Parallel(n_jobs=-1)]: Done 2584 tasks      | elapsed:  6.7min
[Parallel(n_jobs=-1)]: Done 3273 tasks      | elapsed:  8.6min
[Parallel(n_jobs=-1)]: Done 3500 out of 3500 | elapsed:  9.1min finished


RandomizedSearchCV(cv=StratifiedShuffleSplit(n_splits=5, random_state=None, test_size=0.2,
            train_size=None),
                   error_score=nan,
                   estimator=RandomForestClassifier(bootstrap=True,
                                                    ccp_alpha=0.0,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features='auto',
                                                    max_leaf_nodes=None,
                                                    max_samples=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_sample...
                   iid='deprecated', n_iter=700, n_jobs=-1,
 

In [20]:
print(random_model.best_score_)
random_model.best_params_

0.8558659217877095


{'n_estimators': 250,
 'min_samples_split': 4,
 'min_samples_leaf': 4,
 'max_features': 3,
 'max_depth': 5,
 'criterion': 'entropy',
 'bootstrap': False}

In [21]:
best_random = random_model.best_estimator_
pred=best_random.predict(X)
# pred=[int(round(x[0])) for x in pred]
# print(pred[0:10])
from sklearn.metrics import confusion_matrix
print(confusion_matrix(Y,pred))
from sklearn.metrics import accuracy_score
accuracy_score(Y,pred)

[[507  42]
 [ 91 251]]


0.8507295173961841

In [34]:
grid_param= {
    'bootstrap': [False],
    'max_depth': [4,5,6],
    'max_features': [3],
    'min_samples_leaf': [3,4,5],
    'min_samples_split': [4],
    'criterion': ['entropy'],
    'n_estimators': [230,240,250]
}

In [35]:
clf1=RandomForestClassifier()
grid_model = GridSearchCV(estimator = clf1, param_grid = grid_param, 
                          cv = sss, n_jobs = -1, verbose = 2)
grid_model.fit(X,Y)

Fitting 5 folds for each of 27 candidates, totalling 135 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:    6.3s
[Parallel(n_jobs=-1)]: Done 135 out of 135 | elapsed:   24.9s finished


GridSearchCV(cv=StratifiedShuffleSplit(n_splits=5, random_state=None, test_size=0.2,
            train_size=None),
             error_score=nan,
             estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                              class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              max_samples=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf...
                                              n_estimators=100, n_jobs=None,
                                              oob_score=False,
                                              random_state=None, verbose=0,
         

In [36]:
print(grid_model.best_score_)
grid_model.best_params_

0.8469273743016761


{'bootstrap': False,
 'criterion': 'entropy',
 'max_depth': 4,
 'max_features': 3,
 'min_samples_leaf': 5,
 'min_samples_split': 4,
 'n_estimators': 230}

In [37]:
best_grid = grid_model.best_estimator_
pred=best_grid.predict(X)
# pred=[int(round(x[0])) for x in pred]
# print(pred[0:10])
from sklearn.metrics import confusion_matrix
print(confusion_matrix(Y,pred))
from sklearn.metrics import accuracy_score
accuracy_score(Y,pred)

[[508  41]
 [ 95 247]]


0.8473625140291807

In [38]:
preds=grid_model.predict(test.values)
preds=[int(x) for x in preds]
samp=pd.read_csv("../input/titanic/gender_submission.csv")
samp["Survived"]=preds
samp["Survived"].value_counts()

0    270
1    148
Name: Survived, dtype: int64

In [39]:
samp.to_csv("output.csv",index=False)